<a href="https://colab.research.google.com/github/ImagingDataCommons/Cloud-Resources-Workflows/blob/notebooks/Notebooks/Totalsegmentator/PostProcessingTerra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**This notebook can process the data generated by the twoVmWorkflowOnTerra and push to Google Cloud storage buckets.**

The steps are:
- The lz4 compressed DICOM SEG and SR files are downloaded from the Terra Workspace bucket
- They are decompressed by lz4 and moved to a temporary directory
- The uncompressed DICOM SEG and SR files are then pushed to Google Cloud Storage buckets, which can then be imported by DICOM store.
- The process is iterated for each batch

##**Install lz4**

In [ ]:
# %%capture
# !sudo apt-get update \
#   && apt-get install -y --no-install-recommends \
#     lz4\
#   && rm -rf /var/lib/apt/lists/*

##**Authenticate gcloud**

In [ ]:
project_id='my-test-project'

In [ ]:
!gcloud auth login

In [ ]:
# !gcloud config set project $project_id

##**Import packages**

In [ ]:
import pandas as pd
import shutil
import os
from tqdm import tqdm
import traceback

##**Load the terra table containing links to artifacts generated by the twoVmWorkflow**

In [ ]:
data= pd.read_table('twoVM_2023_06_10_00_22.tsv')
data

##**Decompress DICOM SEG files and push to Cloud Storage buckets**

In [ ]:
dicom_seg_download_urls=data['dicomsegAndRadiomicsSR_CompressedFiles'].to_list()
dicom_seg_download_urls

In [ ]:
destination_bucket_name='total_segmentator_nlst_sample_061023'

In [ ]:
for url in tqdm(dicom_seg_download_urls):
    print('processing url:'+url)
    try:
      shutil.rmtree(f'dicom_seg_objects')
      shutil.rmtree(f'itkimage2segimage')
    except OSError:
      pass
    try:
        os.mkdir(f'dicom_seg_objects')
        !gsutil cp {url} . > /dev/null 2>&1
        !lz4 -d --rm dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4 -c | tar --strip-components=1  -xvf - > /dev/null 2>&1
        !find ./itkimage2segimage -name '*.dcm.lz4' -exec mv -t dicom_seg_objects {} + > /dev/null 2>&1
        !lz4 -d -m --rm "dicom_seg_objects"/*.lz4 > /dev/null 2>&1
        !gsutil -m cp -r dicom_seg_objects/* gs://$destination_bucket_name/DICOM_SEGS/ > /dev/null 2>&1
    except Exception as e:
        print(f'Error processing {url}: {e}')
        traceback.print_exc()
shutil.rmtree(f'dicom_seg_objects')
shutil.rmtree(f'itkimage2segimage')

 13%|█▎        | 106/834 [14:19<1:45:13,  8.67s/it]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/d4c69179-ea8b-4d7f-b8ad-979b370ef0e2/call-dicomsegAndRadiomicsSR/dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4


 13%|█▎        | 107/834 [14:28<1:46:09,  8.76s/it]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/9a749850-8e66-4e7d-a21f-c4e3ca23eee9/call-dicomsegAndRadiomicsSR/dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4


 13%|█▎        | 108/834 [14:37<1:46:35,  8.81s/it]

...

100%|█████████▉| 832/834 [2:48:04<00:17,  8.70s/it]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/ddc6b336-be78-49b4-a083-30f579cd3110/call-dicomsegAndRadiomicsSR/dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4


100%|█████████▉| 833/834 [2:48:14<00:08,  8.99s/it]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/a318c435-11aa-4a23-8131-b7fddee7279e/call-dicomsegAndRadiomicsSR/dicomsegAndRadiomicsSR_DICOMsegFiles.tar.lz4


100%|██████████| 834/834 [2:48:23<00:00, 12.12s/it]


##**Decompress DICOM SR files and push to Cloud Storage buckets**

In [ ]:
sr_download_urls=data['structuredReportsDICOM'].to_list()

In [ ]:
series_df = pd.DataFrame(columns=['series_id'])

batch_count = 1  # Counter for batch folders

for url in tqdm(sr_download_urls):
    print('processing url:' + url)
    try:
        shutil.rmtree(f'structuredReportsDICOM')
        shutil.rmtree(f'decompressedStructuredReportsDICOM')
    except OSError:
        pass
    os.mkdir(f'decompressedStructuredReportsDICOM')
    try:
        !gsutil cp {url} . > /dev/null 2>&1
        !lz4 -d --rm structuredReportsDICOM.tar.lz4 -c | tar --strip-components=1 -xvf - > /dev/null 2>&1
        !find ./structuredReportsDICOM -name '*.dcm.lz4' -exec mv -t decompressedStructuredReportsDICOM {} + > /dev/null 2>&1
        !lz4 -d -m --rm "decompressedStructuredReportsDICOM"/*.lz4 > /dev/null 2>&1
        !gsutil -m cp -r decompressedStructuredReportsDICOM/* gs://destination_bucket_name/decompressedStructuredReportsDICOM/batch_{batch_count}/ > /dev/null 2>&1

        # Find all series IDs and add them to the DataFrame
        series_ids = [filename.split('_')[0] for filename in os.listdir('decompressedStructuredReportsDICOM')]
        url_series_df = pd.DataFrame({'series_id': series_ids})

        # Append the current DataFrame to the main DataFrame
        series_df = pd.concat([series_df, url_series_df], ignore_index=True)

    except Exception as e:
        print(f'Error processing {url}: {e}')
        traceback.print_exc()

#the below steps are for importing dicom files to dicom store from command line.
    # try:
    #     # Upload shape_sr.dcm files in the batch
    #     gcs_uri = f"gs://destination_bucket_name/decompressedStructuredReportsDICOM/batch_{batch_count}/*.dcm"
    #     !gcloud healthcare dicom-stores import gcs 10k-series --dataset=total_segmentator_nlst_sample_061023 --location=us-central1 --gcs-uri={gcs_uri}
    # except Exception as e:
    #     print(f'Error processing {url}: {e}')
    #     traceback.print_exc()

    # Increment the batch counter
    batch_count += 1


  0%|          | 0/834 [00:00<?, ?it/s]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/09254959-5ee1-48ed-863c-895e385abef0/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4
Request issued for: [10k-series]
Waiting for operation [projects/idc-external-025/locations/us-central1/datasets
/total_segmentator_nlst_sample_061023/operations/11509072047540011009] to compl
ete...done.                                                                    
name: projects/idc-external-025/locations/us-central1/datasets/total_segmentator_nlst_sample_061023/dicomStores/10k-series


  0%|          | 1/834 [00:12<2:58:23, 12.85s/it]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/cee58ab9-6c35-48b2-99f7-746be2966cfd/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4
Request issued for: [10k-series]
Waiting for operation [projects/idc-external-025/locations/us-central1/datasets
/total_segmentator_nlst_sample_061023/operations/7069327554385018881] to comple
te...done.                                                                     
name: projects/idc-external-025/locations/us-central1/datasets/total_segmentator_nlst_sample_061023/dicomStores/10k-series


  0%|          | 2/834 [00:29<3:29:46, 15.13s/it]

...

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/2ccf9f05-a2d2-408d-b48c-b5c92f09c8b6/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4
Request issued for: [10k-series]
Waiting for operation [projects/idc-external-025/locations/us-central1/datasets
/total_segmentator_nlst_sample_061023/operations/2063809768107016193] to comple
te...done.                                                                     
name: projects/idc-external-025/locations/us-central1/datasets/total_segmentator_nlst_sample_061023/dicomStores/10k-series


 45%|████▌     | 379/834 [1:40:48<1:39:11, 13.08s/it]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/159ef78b-f7c1-4f7e-bdf7-36f2aa452ac3/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4
Request issued for: [10k-series]
Waiting for operation [projects/idc-external-025/locations/us-central1/datasets
/total_segmentator_nlst_sample_061023/operations/12222256967678164993] to compl
ete...done.                                                                    
name: projects/idc-external-025/locations/us-central1/datasets/total_segmentator_nlst_sample_061023/dicomStores/10k-series


 46%|████▌     | 380/834 [1:41:00<1:35:34, 12.63s/it]

processing url:gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/c3c7ec98-e7a3-4cdb-8576-76720b002ef9/TotalSegmentator/e768dec4-a47b-4bdf-8546-21ae30c88827/call-dicomsegAndRadiomicsSR/structuredReportsDICOM.tar.lz4
Request issued for: [10k-series]
Waiting for operation [projects/idc-external-025/locations/us-central1/datasets
/total_segmentator_nlst_sample_061023/operations/11043014254213988353] to compl
ete...⠏                                                                        

In [ ]:
shutil.rmtree(f'structuredReportsDICOM')